In [1]:
import socket
import torch
import torchvision.transforms as transforms
from VGG16 import VGG16
from PIL import Image

In [2]:
labels = ['Paper', 'Rock', 'Scissor']
path = ''
def readImage(path:str):
    img = Image.open(str(path)).convert('RGB')
    return img
def preprocessImage(img):
    show_process = transforms.Compose([
        transforms.CenterCrop(224)
    ])
    show_process(img).show()
    preprocess = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # 标准化
    ])
    tensor = preprocess(img)
    return tensor

In [3]:
# init module
module = torch.load('module.pkl', map_location=torch.device('cpu'))
module.eval()

def predict(img_path:str) -> (torch.tensor, str):
    tensor = preprocessImage(readImage(img_path))
    tensor = torch.unsqueeze(tensor, dim=0)
    with torch.no_grad():
        _, outputs = module(tensor)
        _, predicted = torch.max(outputs.data, 1)
    print('predicted:', labels[predicted])
    return outputs.data, labels[predicted]

In [4]:
# communication
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(('127.0.0.1', 6666))
    s.listen(1)
    c, addr = s.accept()
    with c:
        print(addr, "connected.")
        while True:
            data = c.recv(1024)
            # exit signal
            if not data or data == b'exit\n':
                break
            data = bytes.decode(data)
            # remove \n
            if data[-1] == '\n':
                data = data[0:-1]
            output, prediction = predict(data)
            c.sendall(str.encode(prediction) + b'\n')
print('Connection closed.')

('127.0.0.1', 1078) connected.
predicted: Paper
predicted: Rock
predicted: Rock
predicted: Scissor
predicted: Scissor
predicted: Rock
predicted: Rock


KeyboardInterrupt: 